In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optimizers
import torchvision
import os
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.utils.data
import torchsummary
from PIL import Image
from matplotlib import cm

In [2]:
outf = 'result_3_2-CGAN'

In [3]:
try:
    os.makedirs(outf, exist_ok=True)
except OSError as error: 
    print(error)
    pass

In [4]:
workers=2
batch_size=50
nz=100
nch_g=128
nch_d=128
n_epoch=10
lr=0.0002
beta1=0.5

In [ ]:
root = os.path.join('data', 'mnist')
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,))])
mnist_train =torchvision.datasets.MNIST(root=root,
                                      download=True,
                                      train=True,
                                      transform=transform)
dataloader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
                                         shuffle=True, num_workers=int(workers))

# 学習に使用するデバイスを得る。可能ならGPUを使用する
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

In [6]:
class Generator(nn.Module):
    """
    torch.nn.ConvTranspose2d(in_channels, out_channels, 
    kernel_size, stride=1, padding=0, 
    output_padding=0, groups=1, bias=True, 
    dilation=1, padding_mode='zeros')
    """
    def __init__(self, nz=100, nch_g=128, nch=1):
        super(Generator,self).__init__()
        self.layers=nn.ModuleDict({
            'layer0':nn.Sequential(
            nn.ConvTranspose2d(nz,nch_g*4,3,1,0),
            nn.BatchNorm2d(nch_g*4),
            nn.ReLU()),
            'layer1':nn.Sequential(
            nn.ConvTranspose2d(nch_g*4,nch_g*2,3,2,0),
            nn.BatchNorm2d(nch_g*2),
            nn.ReLU()),
            'layer2':nn.Sequential(
            nn.ConvTranspose2d(nch_g*2,nch_g,4,2,1),
            nn.BatchNorm2d(nch_g),
            nn.ReLU()),
            'layer3':nn.Sequential(
            nn.ConvTranspose2d(nch_g,nch,4,2,1),
            nn.Tanh())
        })
    
    def forward(self,z):
        for layer in self.layers.values():
            z=layer(z)
        return z
    

In [5]:
class Discriminator(nn.Module):
    def __init__(self, nch=1, nch_d=128):
        
        super(Discriminator, self).__init__()

    
        self.layers = nn.ModuleDict({
            'layer0': nn.Sequential(
                nn.Conv2d(nch, nch_d, 4, 2, 1),     
                nn.LeakyReLU(negative_slope=0.2)   
            ), 
            'layer1': nn.Sequential(
                nn.Conv2d(nch_d, nch_d * 2, 4, 2, 1),
                nn.BatchNorm2d(nch_d * 2),
                nn.LeakyReLU(negative_slope=0.2)
            ), 
            'layer2': nn.Sequential(
                nn.Conv2d(nch_d * 2, nch_d * 4, 3, 2, 0),
                nn.BatchNorm2d(nch_d * 4),
                nn.LeakyReLU(negative_slope=0.2)
            ), 
            'layer3': nn.Sequential(
                nn.Conv2d(nch_d * 4, 1, 3, 1, 0),
                nn.Sigmoid()   
            )    
            
        })

    def forward(self, x):
    
        for layer in self.layers.values():  
            x = layer(x)
        return x.squeeze()  

In [7]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:          
        m.weight.data.normal_(0.0, 0.02)
        m.bias.data.fill_(0)
    elif classname.find('Linear') != -1:      
        m.bias.data.fill_(0)
    elif classname.find('BatchNorm') != -1:   
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)